In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('pre').getOrCreate()

22/07/07 12:56:40 WARN Utils: Your hostname, exodus resolves to a loopback address: 127.0.1.1; using 10.100.229.37 instead (on interface wlp4s0)
22/07/07 12:56:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/07 12:56:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option('header', 'true').option('inferSchema', 'true').csv('./OutEnv-hour.csv')

In [4]:
df.count()

8761

In [5]:
df = df.na.drop(subset=["Timestamp","OutEnv_OutdoorAmbTemp"])
df_rdd = df.rdd

In [6]:
df.count()

8672

In [7]:
from pyspark.sql import Row
def pydf_to_str(row, col):
    rd = row.asDict()
    rd[col] = str(rd[col])[0:19]
    nrow = Row(**rd)
    return nrow
def pydf_to_int(row, col):
    rd = row.asDict()
    rd[col] = round(rd[col],0)
    nrow = Row(**rd)
    return nrow


In [8]:
df_rdd = df_rdd.map(lambda x:pydf_to_str(x, 'Timestamp'))
df_rdd = df_rdd.filter(lambda x:x['Timestamp'] >= '2013-07-01').filter(lambda x:x['Timestamp'] <'2013-08-01')

In [9]:
df_rdd.take(1)

[Row(Timestamp='2013-07-01 00:00:00', OutEnv_OutdoorAmbTemp=23.073361342105283, OutEnv_RooftopWindDirection=194.72881355932202, OutEnv_RooftopWindSpeed=2.15593220338983)]

In [10]:
df_rdd = df_rdd.map(lambda x:pydf_to_int(x, 'OutEnv_OutdoorAmbTemp'))

In [11]:
df_rdd.take(1)

[Row(Timestamp='2013-07-01 00:00:00', OutEnv_OutdoorAmbTemp=23.0, OutEnv_RooftopWindDirection=194.72881355932202, OutEnv_RooftopWindSpeed=2.15593220338983)]

In [12]:
import math
df_rdd2 = df_rdd.map(lambda x: 
    (x[1], int(str(x[0])[8:10]))
)
df_rdd2.collect()

[(23.0, 1),
 (23.0, 1),
 (22.0, 1),
 (22.0, 1),
 (22.0, 1),
 (22.0, 1),
 (22.0, 1),
 (22.0, 1),
 (23.0, 1),
 (24.0, 1),
 (24.0, 1),
 (24.0, 1),
 (23.0, 1),
 (24.0, 1),
 (26.0, 1),
 (27.0, 1),
 (27.0, 1),
 (27.0, 1),
 (27.0, 1),
 (26.0, 1),
 (26.0, 1),
 (25.0, 1),
 (25.0, 1),
 (24.0, 1),
 (24.0, 2),
 (24.0, 2),
 (24.0, 2),
 (24.0, 2),
 (23.0, 2),
 (23.0, 2),
 (23.0, 2),
 (24.0, 2),
 (24.0, 2),
 (25.0, 2),
 (25.0, 2),
 (26.0, 2),
 (26.0, 2),
 (28.0, 2),
 (27.0, 2),
 (26.0, 2),
 (25.0, 2),
 (25.0, 2),
 (24.0, 2),
 (23.0, 2),
 (23.0, 2),
 (23.0, 2),
 (23.0, 2),
 (23.0, 2),
 (23.0, 3),
 (23.0, 3),
 (22.0, 3),
 (22.0, 3),
 (22.0, 3),
 (22.0, 3),
 (22.0, 3),
 (22.0, 3),
 (22.0, 3),
 (23.0, 3),
 (23.0, 3),
 (24.0, 3),
 (26.0, 3),
 (27.0, 3),
 (29.0, 3),
 (29.0, 3),
 (26.0, 3),
 (26.0, 3),
 (26.0, 3),
 (26.0, 3),
 (25.0, 3),
 (25.0, 3),
 (25.0, 3),
 (24.0, 3),
 (24.0, 4),
 (23.0, 4),
 (23.0, 4),
 (23.0, 4),
 (23.0, 4),
 (23.0, 4),
 (24.0, 4),
 (24.0, 4),
 (25.0, 4),
 (25.0, 4),
 (26.0, 4),
 (27

In [13]:
df_rdd3 = df_rdd2.groupByKey()
df_rdd3.collect()

[(23.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaecbe0>),
 (22.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf36d0>),
 (24.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3280>),
 (26.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3100>),
 (27.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3190>),
 (25.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf30a0>),
 (28.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3370>),
 (29.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3490>),
 (30.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf34c0>),
 (31.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3070>),
 (32.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf39a0>),
 (21.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3a00>),
 (20.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3a60>),
 (19.0, <pyspark.resultiterable.ResultIterable at 0x7f09beaf3ac0>),
 (33.0, <pyspark.resultiterable.ResultIterable a

In [14]:
def make_bitmap(x):
    s = list(31*"0")
    for i in x:
        s[i-1] = "1"
    return "".join(s)

In [15]:
df_rdd3 = df_rdd3.map(lambda x:(x[0], x[1], make_bitmap(x[1])))
df_rdd3.collect()

[(23.0,
  '1111101111111010000011111111111'),
 (22.0,
  '1010001100111000000001111111111'),
 (24.0,
  '1111111111101110000111011111111'),
 (26.0,
  '1111111111101111111111110111111'),
 (27.0,
  '1111111110101101111011110111110'),
 (25.0,
  '1111111111101111110111110101111'),
 (28.0,
  '0101111111101111111111110111000'),
 (29.0,
  '0011111111001111111101100000000'),
 (30.0,
  '0001111101000111111111100000000'),
 (31.0,
  '0001111001000111110110000000000'),
 (32.0,
  '0000010000000011111100000000000'),
 (21.0,
  '0000001100011000000000001111101'),
 (20.0,
  '0000000000011000000000001011111'),
 (19.0,
  '0000000000010000000000001100111'),
 (33.0,
  '0000000000000011111100000000000'),
 (34.0,
  '0000000000000010011000000000000'),
 (35.0,
  '0000000000000000011000000000000'),
 (18.0,
  '0000000000000000000000001100110'),
 (17.0,
  '0000000000000000000000001100110'),
 (16.0,
  '0000000000000000000000001000010'),
 (15.0,
  '0000000000000000000000001000000')]

In [16]:
def single_supp(event):
    supp = 0
    bitmap = event[2]
    for bit_i in range(len(bitmap)):
        if bitmap[bit_i] == '1':
            supp = supp + 1
    return supp

df_rdd3.map(single_supp).collect()

[24, 17, 25, 29, 25, 27, 24, 18, 16, 12, 7, 10, 8, 6, 6, 3, 2, 4, 4, 2, 1]

In [17]:
sigma = 5
support_rdd = df_rdd3.filter(lambda x: single_supp(x) >= sigma)
support_rdd.collect()

[(23.0,
  '1111101111111010000011111111111'),
 (22.0,
  '1010001100111000000001111111111'),
 (24.0,
  '1111111111101110000111011111111'),
 (26.0,
  '1111111111101111111111110111111'),
 (27.0,
  '1111111110101101111011110111110'),
 (25.0,
  '1111111111101111110111110101111'),
 (28.0,
  '0101111111101111111111110111000'),
 (29.0,
  '0011111111001111111101100000000'),
 (30.0,
  '0001111101000111111111100000000'),
 (31.0,
  '0001111001000111110110000000000'),
 (32.0,
  '0000010000000011111100000000000'),
 (21.0,
  '0000001100011000000000001111101'),
 (20.0,
  '0000000000011000000000001011111'),
 (19.0,
  '0000000000010000000000001100111'),
 (33.0,
  '0000000000000011111100000000000')]

In [18]:
def supp(bitmap_A, bitmap_B):
    ocurrence = 0
    for bit_i in range(len(bitmap_A)):
        if bitmap_A[bit_i] == '1' and bitmap_B[bit_i] == '1':
            ocurrence = ocurrence + 1
            new_bitmap += '1'
        else:
            new_bitmap += '0'